In [ ]:
#!/usr/bin/env python
#coding: utf-8


In [ ]:

#Importing modules

get_ipython().system('pip install geopandas')
get_ipython().system('pip install scikit-learn')
get_ipython().system('pip install datetime')
import geopandas as gpd
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
from datetime import datetime
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score




In [ ]:
#Labelling outputs
change_type_map = {'Demolition': 0, 'Road': 1, 'Residential': 2, 'Commercial': 3, 'Industrial': 4,
       'Mega Projects': 5}

## Read jsons

train_df1 = gpd.read_file('train.geojson', index_col='index')
test_df1 = gpd.read_file('test.geojson', index_col='index')





In [ ]:
#Creating an independent copy of the original dataset to work on.
train_df = train_df1.copy()
test_df = test_df1.copy()



 # Feature engineering

In [ ]:

# One-Hot Encoding of urban-type:
types_de_zones_urbaines = ['Sparse Urban', 'Rural', 'Dense Urban', 'Industrial','Urban Slum','N,A']
# Création of a feature for each element of urban-type
for zone in types_de_zones_urbaines:
    train_df[zone] = train_df.urban_type.map(lambda x: 1 if zone in x else 0)
    test_df[zone] = test_df.urban_type.map(lambda x: 1 if zone in x else 0)

In [ ]:
#Label Encoding with number proportional to the degre of construction 
types_de_status= {'Construction Started':0,'Land Cleared':1 , 'Prior Construction':2,'Operational':3,'Construction Midway':4,'Greenland':5,'Excavation':6,'Construction Done':7, 'Materials Dumped':8, 'Materials Introduced':9,None:10}

train_df.change_status_date0 = train_df.change_status_date0.map(types_de_status)
train_df.change_status_date1 = train_df.change_status_date1.map(types_de_status)
train_df.change_status_date2 = train_df.change_status_date2.map(types_de_status)
train_df.change_status_date3 = train_df.change_status_date3.map(types_de_status)
train_df.change_status_date4 = train_df.change_status_date4.map(types_de_status)

test_df.change_status_date0 = test_df.change_status_date0.map(types_de_status)
test_df.change_status_date1 = test_df.change_status_date1.map(types_de_status)
test_df.change_status_date2 = test_df.change_status_date2.map(types_de_status)
test_df.change_status_date3 = test_df.change_status_date3.map(types_de_status)
test_df.change_status_date4 = test_df.change_status_date4.map(types_de_status)



In [ ]:
#One-hot endoding on geography_type

train_y = train_df['change_type'].apply(lambda x: change_type_map[x])


types_de_geographies = []
for str in train_df.geography_type :
    values = str.split(',')
    for k in values:
        if k not in types_de_geographies:
            types_de_geographies.append(k)
            
types_de_geographies.remove('N')
types_de_geographies.remove('A')

for geographie in types_de_geographies:
    train_df[geographie] = train_df.geography_type.map(lambda x: 1 if geographie in x else 0)
    test_df[geographie] = test_df.geography_type.map(lambda x: 1 if geographie in x else 0)




In [ ]:

#conversion of dates
for i in range(0,5):
    print(train_df[f'date{i}'])
    train_df[f'date{i}'] = pd.to_datetime(train_df[f'date{i}'], format='%d-%m-%Y')
    test_df[f'date{i}'] = pd.to_datetime(test_df[f'date{i}'], format='%d-%m-%Y')

# Conversion of dates in timestamps Unix (time since january 1st 2000)
    train_df[f'date{i}'] = (train_df[f'date{i}'] - pd.Timestamp("2000-01-01")) // pd.Timedelta('1s')/(10**9)
    test_df[f'date{i}'] = (test_df[f'date{i}'] - pd.Timestamp("2000-01-01")) // pd.Timedelta('1s')/(10**9)

# Time difference

train_df['diff_date1_date0'] = train_df['date1'] - train_df['date0']
train_df['diff_date2_date1'] = train_df['date2'] - train_df['date1']
train_df['diff_date3_date2'] = train_df['date3'] - train_df['date2']
train_df['diff_date4_date3'] = train_df['date4'] - train_df['date3']

test_df['diff_date1_date0'] = test_df['date1'] - test_df['date0']
test_df['diff_date2_date1'] = test_df['date2'] - test_df['date1']
test_df['diff_date3_date2'] = test_df['date3'] - test_df['date2']
test_df['diff_date4_date3'] = test_df['date4'] - test_df['date3']

#Creation of features about the geometry

train_df['area'] = train_df[['geometry']].area
train_df['perimeter'] = train_df['geometry'].length
train_df['area/perimeter'] = train_df[['geometry']].area / (train_df['geometry'].length)
train_df['area/perimeter**2'] = train_df[['geometry']].area / (train_df['geometry'].length)**2

test_df['area'] = test_df[['geometry']].area
test_df['perimeter'] = test_df['geometry'].length
test_df['area/perimeter'] = test_df[['geometry']].area / (test_df['geometry'].length)
test_df['area/perimeter**2'] = test_df[['geometry']].area / (test_df['geometry'].length)**2


#  Colonne de niveaux de gris
#train_df['grayscale_date1'] = 0.2989 * train_df['img_red_mean_date1'] + 0.5870 * train_df['img_green_mean_date1'] + 0.1140 * train_df['img_blue_mean_date1']
#train_df['grayscale_date2'] = 0.2989 * train_df['img_red_mean_date2'] + 0.5870 * train_df['img_green_mean_date2'] + 0.1140 * train_df['img_blue_mean_date2']
#train_df['grayscale_date3'] = 0.2989 * train_df['img_red_mean_date3'] + 0.5870 * train_df['img_green_mean_date3'] + 0.1140 * train_df['img_blue_mean_date3']
#train_df['grayscale_date4'] = 0.2989 * train_df['img_red_mean_date4'] + 0.5870 * train_df['img_green_mean_date4'] + 0.1140 * train_df['img_blue_mean_date4']
#train_df['grayscale_date5'] = 0.2989 * train_df['img_red_mean_date5'] + 0.5870 * train_df['img_green_mean_date5'] + 0.1140 * train_df['img_blue_mean_date5']

#test_df['grayscale_date1'] = 0.2989 * test_df['img_red_mean_date1'] + 0.5870 * test_df['img_green_mean_date1'] + 0.1140 * test_df['img_blue_mean_date1']
#test_df['grayscale_date2'] = 0.2989 * test_df['img_red_mean_date2'] + 0.5870 * test_df['img_green_mean_date2'] + 0.1140 * test_df['img_blue_mean_date2']
#test_df['grayscale_date3'] = 0.2989 * test_df['img_red_mean_date3'] + 0.5870 * test_df['img_green_mean_date3'] + 0.1140 * test_df['img_blue_mean_date3']
#test_df['grayscale_date4'] = 0.2989 * test_df['img_red_mean_date4'] + 0.5870 * test_df['img_green_mean_date4'] + 0.1140 * test_df['img_blue_mean_date4']
#test_df['grayscale_date5'] = 0.2989 * test_df['img_red_mean_date5'] + 0.5870 * test_df['img_green_mean_date5'] + 0.1140 * test_df['img_blue_mean_date5']


#  Différence entre les canaux et les niveaux de gris pour chaque date
for date in range(1, 5):
    # différence pour chaque canal RVB
    train_df[f'diff_red_date{date}'] = train_df[f'img_red_mean_date{date+1}'] - train_df[f'img_red_mean_date{date}']
    train_df[f'diff_green_date{date+1}'] = train_df[f'img_green_mean_date{date+1}'] - train_df[f'img_green_mean_date{date}']
    train_df[f'diff_blue_date{date+1}'] = train_df[f'img_blue_mean_date{date+1}'] - train_df[f'img_blue_mean_date{date}']
    #train_df[f'diff_gray_date{date+1}'] = train_df[f'grayscale_date{date+1}'] - train_df[f'grayscale_date{date}']
    train_df[f'diff_red_std_date{date}'] = train_df[f'img_red_std_date{date+1}'] - train_df[f'img_red_std_date{date}']
    train_df[f'diff_green_std_date{date+1}'] = train_df[f'img_green_std_date{date+1}'] - train_df[f'img_green_std_date{date}']
    train_df[f'diff_blue_std_date{date+1}'] = train_df[f'img_blue_std_date{date+1}'] - train_df[f'img_blue_std_date{date}']
    
    test_df[f'diff_red_date{date}'] = test_df[f'img_red_mean_date{date+1}'] - test_df[f'img_red_mean_date{date}']
    test_df[f'diff_green_date{date+1}'] = test_df[f'img_green_mean_date{date+1}'] - test_df[f'img_green_mean_date{date}']
    test_df[f'diff_blue_date{date+1}'] = test_df[f'img_blue_mean_date{date+1}'] - test_df[f'img_blue_mean_date{date}']
    #test_df[f'diff_gray_date{date+1}'] = test_df[f'grayscale_date{date+1}'] - test_df[f'grayscale_date{date}']
    test_df[f'diff_red_std_date{date}'] = train_df[f'img_red_std_date{date+1}'] - train_df[f'img_red_std_date{date}']
    test_df[f'diff_green_std_date{date+1}'] = train_df[f'img_green_std_date{date+1}'] - train_df[f'img_green_std_date{date}']
    test_df[f'diff_blue_std_date{date+1}'] = train_df[f'img_blue_std_date{date+1}'] - train_df[f'img_blue_std_date{date}']





In [ ]:
# Suppression of features
    
colonnes_a_supprimer = ['urban_type', 'geography_type','change_type',
                        'geometry','index','date0','date1','date2','date3','date4']


train_df.drop(colonnes_a_supprimer, axis=1, inplace=True)
colonnes_a_supprimer.remove('change_type')

test_df.drop(colonnes_a_supprimer, axis=1, inplace=True)


In [ ]:
#  NA replaced by the mean of the feature
for colonne in train_df.columns[train_df.isna().any()]:
    train_df[colonne].fillna(train_df[colonne].mean() ,inplace = True)

for colonne in test_df.columns[test_df.isna().any()]:
    test_df[colonne].fillna(test_df[colonne].mean() ,inplace = True)


In [ ]:

train_df = train_df.astype(float)
test_df = test_df.astype(float)


In [ ]:
# ## Normalization of train

# Mean of each column
mean_data = train_df.mean()

# écart type of each column
std_data = train_df.std()

# Normalization of each column

normalized_train_df = train_df.apply(lambda x: (x - mean_data)/std_data , axis =1)

In [ ]:
# ## Normalization of test

# Calculer la moyenne de chaque colonne de la donnée de test
mean_data = test_df.mean()

# Calculer l'écart type de chaque colonne
std_data = test_df.std()

# Normaliser chaque colonne indépendamment

normalized_test_df = test_df.apply(lambda x: (x - mean_data)/std_data , axis =1)




In [ ]:
# ##  LDA
train_df_LDA = train_df
train_y_LDA = train_y

lda = LDA(n_components=5)  # Choisir le nombre de composantes
X_trainlda = lda.fit_transform(train_df_LDA, train_y_LDA)
X_trainlda_df = pd.DataFrame(X_trainlda)

In [ ]:
#LDA
X_testlda = lda.transform(test_df)
X_testlda_df = pd.DataFrame(X_testlda)

In [ ]:
# ## PCA
train_df_PCA = normalized_train_df

#PCA on train data
pca = PCA().fit(train_df_PCA)
variance_cumulee = np.cumsum(pca.explained_variance_ratio_)
# 95% of variance kept
num_components = np.where(variance_cumulee >= 0.95)[0][0] + 1
pca = PCA(n_components=num_components)
df_PCA = pca.fit_transform(train_df_PCA)
df_PCA = pd.DataFrame(df_PCA)
df_PCA.to_csv("TrainDataAfterPCA-95%.csv", index=False)





In [ ]:

test_df_PCA = normalized_test_df
df_test_PCA = pca.transform(test_df_PCA)
df_test_PCA = pd.DataFrame(df_test_PCA)


In [ ]:

# Initialisation 
model =KNeighborsClassifier(n_neighbors=4)

# Définir le nombre de plis pour la validation croisée
num_folds = 5

# Initialiser la validation croisée k-fold
kfold = KFold(n_splits=num_folds)

# Effectuer la validation croisée en utilisant la fonction cross_val_score
scores = cross_val_score(model,train_df, train_y, cv=kfold)

# Afficher les scores de chaque pli
print("Scores de chaque pli:", scores)
#afficher la moyenne
mean_score = scores.mean()
print("Score moyen de validation croisée:", mean_score)





In [ ]:


# ### Prévision 

neigh = KNeighborsClassifier(n_neighbors=4)
neigh.fit(df_PCA, train_y) #ici c est avec PCA mais on peut aussi faire avec LDA
pred_y_KNN = neigh.predict(df_test_PCA)





In [ ]:




answer_df = pd.DataFrame(pred_y_KNN, columns = ['change_type'])

answer_df.index.name = 'Id'

answer_df.to_csv('answer.csv')


In [ ]:

# Initialiser le modèle 
model =SVC(kernel='rbf')

# Définir le nombre de plis pour la validation croisée
num_folds = 5

# Initialiser la validation croisée k-fold
kfold = KFold(n_splits=num_folds)

# Effectuer la validation croisée en utilisant la fonction cross_val_score
scores = cross_val_score(model,df_PCA, train_y, cv=kfold)

# Afficher les scores de chaque pli
print("Scores de chaque pli:", scores)
#afficher la moyenne
mean_score = scores.mean()
print("Score moyen de validation croisée:", mean_score)





In [ ]:
# ### LDA-cross validation




# Initialiser le modèle 
model =SVC(kernel='rbf')

# Définir le nombre de plis pour la validation croisée
num_folds = 5

# Initialiser la validation croisée k-fold
kfold = KFold(n_splits=num_folds)

# Effectuer la validation croisée en utilisant la fonction cross_val_score
scores = cross_val_score(model,X_trainlda_df, train_y, cv=kfold)

# Afficher les scores de chaque pli
print("Scores de chaque pli:", scores)
#afficher la moyenne
mean_score = scores.mean()
print("Score moyen de validation croisée:", mean_score)



In [ ]:
svm_model = SVC(kernel='rbf')
svm_model.fit(df_PCA, train_y) # a modifier selon les données que l'on souhaite

y_pred = svm_model.predict(df_test_PCA)

y_train_pred = svm_model.predict(df_PCA)

# Calculer la précision du modèle sur les données d'entraînement
train_accuracy = accuracy_score(train_y, y_train_pred)
print("Précision du modèle SVM sur les données d'entraînement:", train_accuracy)



In [ ]:
answer_SVM = pd.DataFrame(y_pred, columns = ['change_type'])

answer_SVM.index.name = 'Id'

answer_SVM.to_csv('answer.csv')



In [ ]:

# Initialiser le modèle de forêt aléatoire
model =RandomForestClassifier(n_estimators=50)

# Définir le nombre de plis pour la validation croisée
num_folds = 5

# Initialiser la validation croisée k-fold
kfold = KFold(n_splits=num_folds)

# Effectuer la validation croisée en utilisant la fonction cross_val_score
scores = cross_val_score(model,train_df, train_y, cv=kfold)

# Afficher les scores de chaque pli
print("Scores de chaque pli:", scores)
#afficher la moyenne
mean_score = scores.mean()
print("Score moyen de validation croisée:", mean_score)


# ### Prediction


In [ ]:
rf_model = RandomForestClassifier(n_estimators=50)  # Vous pouvez ajuster le nombre d'estimateurs selon votre besoin
rf_model.fit(train_df, train_y)
y_pred_train=rf_model.predict(train_df)

# Prédire les étiquettes des données de test
y_label_train=train_y
y_pred_forest = rf_model.predict(test_df)

# Calculer la précision du modèle sur les données de test
accuracy = accuracy_score(y_pred_train, y_label_train)
print("Précision du modèle Random Forest sur les données de test:", accuracy)

#train_accuracy = accuracy_score(y_label_train, y_pred_forest)
print("Précision du modèle Random Forest sur les données d'entraînement:", train_accuracy)





answer_RandFor = pd.DataFrame(y_pred_forest, columns = ['change_type'])

answer_RandFor.index.name = 'Id'

answer_RandFor.to_csv('answer_Rand.csv')

In [ ]:
# In[ ]:


rf_model = RandomForestClassifier(n_estimators=50)  # Vous pouvez ajuster le nombre d'estimateurs selon votre besoin
rf_model.fit(train_df, train_y)
y_pred_train=rf_model.predict(train_df)

# Prédire les étiquettes des données de test
y_label_train=train_y
y_pred_forest = rf_model.predict(test_df)

# Calculer la précision du modèle sur les données de test
accuracy = accuracy_score(y_pred_train, y_label_train)
print("Précision du modèle Random Forest sur les données de test:", accuracy)

#train_accuracy = accuracy_score(y_label_train, y_pred_forest)
print("Précision du modèle Random Forest sur les données d'entraînement:", train_accuracy)





answer_RandFor = pd.DataFrame(y_pred_forest, columns = ['change_type'])

answer_RandFor.index.name = 'Id'

answer_RandFor.to_csv('answer_Rand.csv')